In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("/content/nbaallelo.csv", on_bad_lines='skip')


<ipython-input-2-abaff6db6f78>:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/nbaallelo.csv", on_bad_lines='skip')


In [3]:

df

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010trh,nba,0,1947,11/1/1946,1,0,trh,huskies,...,40.294830,nyk,knicks,68,1300.0000,1306.7233,h,l,0.640065,NaN
1,1,194611010trh,nba,1,1947,11/1/1946,1,0,nyk,knicks,...,41.705170,trh,huskies,66,1300.0000,1293.2767,a,w,0.359935,NaN
2,2,194611020chs,nba,0,1947,11/2/1946,1,0,chs,stags,...,42.012257,nyk,knicks,47,1306.7233,1297.0712,h,w,0.631101,NaN
3,2,194611020chs,nba,1,1947,11/2/1946,2,0,nyk,knicks,...,40.692783,chs,stags,63,1300.0000,1309.6521,a,l,0.368899,NaN
4,3,194611020dtf,nba,0,1947,11/2/1946,1,0,dtf,falcons,...,38.864048,wsc,capitols,50,1300.0000,1320.3811,h,l,0.640065,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125779,63155,201506110cle,nba,0,2015,6/11/2015,100,1,cle,cavaliers,...,60.309792,gsw,warriors,103,1790.9591,1809.9791,h,l,0.546572,NaN
125780,63156,201506140gsw,nba,0,2015,6/14/2015,102,1,gsw,warriors,...,68.013329,cle,cavaliers,91,1704.3949,1700.7391,h,w,0.765565,NaN
125781,63156,201506140gsw,nba,1,2015,6/14/2015,101,1,cle,cavaliers,...,60.010067,gsw,warriors,104,1809.9791,1813.6349,a,l,0.234435,NaN
125782,63157,201506170cle,nba,0,2015,6/16/2015,102,1,cle,cavaliers,...,59.290245,gsw,warriors,105,1813.6349,1822.2881,h,l,0.481450,NaN


In [4]:
data = df.drop(['game_id', 'notes','date_game'], axis=1)

In [5]:
print(data.isnull().sum())

gameorder        0
lg_id            0
_iscopy          0
year_id          0
seasongame       0
is_playoffs      0
team_id          0
fran_id          0
pts              0
elo_i            0
elo_n            0
win_equiv        0
opp_id           0
opp_fran         0
opp_pts          0
opp_elo_i        0
opp_elo_n        0
game_location    0
game_result      0
forecast         0
dtype: int64


In [6]:
data

,gameorder,lg_id,_iscopy,year_id,seasongame,is_playoffs,team_id,fran_id,pts,elo_i,elo_n,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast
0,1,nba,0,1947,1,0,trh,huskies,66,1300.0000,1293.2767,40.294830,nyk,knicks,68,1300.0000,1306.7233,h,l,0.640065
1,1,nba,1,1947,1,0,nyk,knicks,68,1300.0000,1306.7233,41.705170,trh,huskies,66,1300.0000,1293.2767,a,w,0.359935
2,2,nba,0,1947,1,0,chs,stags,63,1300.0000,1309.6521,42.012257,nyk,knicks,47,1306.7233,1297.0712,h,w,0.631101
3,2,nba,1,1947,2,0,nyk,knicks,47,1306.7233,1297.0712,40.692783,chs,stags,63,1300.0000,1309.6521,a,l,0.368899
4,3,nba,0,1947,1,0,dtf,falcons,33,1300.0000,1279.6189,38.864048,wsc,capitols,50,1300.0000,1320.3811,h,l,0.640065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125779,63155,nba,0,2015,100,1,cle,cavaliers,82,1723.4149,1704.3949,60.309792,gsw,warriors,103,1790.9591,1809.9791,h,l,0.546572
125780,63156,nba,0,2015,102,1,gsw,warriors,104,1809.9791,1813.6349,68.013329,cle,cavaliers,91,1704.3949,1700.7391,h,w,0.765565
125781,63156,nba,1,2015,101,1,cle,cavaliers,91,1704.3949,1700.7391,60.010067,gsw,warriors,104,1809.9791,1813.6349,a,l,0.234435
125782,63157,nba,0,2015,102,1,cle,cavaliers,97,1700.7391,1692.0859,59.290245,gsw,warriors,105,1813.6349,1822.2881,h,l,0.481450


In [7]:


le = LabelEncoder()

data['lg_id'] = le.fit_transform(data['lg_id'])
data['team_id'] = le.fit_transform(data['team_id'])
data['fran_id'] = le.fit_transform(data['fran_id'])
data['opp_id'] = le.fit_transform(data['opp_id'])
data['opp_fran'] = le.fit_transform(data['opp_fran'])
data['game_location'] = le.fit_transform(data['game_location'])
data['game_result'] = le.fit_transform(data['game_result'])

data


,gameorder,lg_id,_iscopy,year_id,seasongame,is_playoffs,team_id,fran_id,pts,elo_i,elo_n,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast
0,1,1,0,1947,1,0,93,17,66,1300.0000,1293.2767,40.294830,63,22,68,1300.0000,1306.7233,1,0,0.640065
1,1,1,1,1947,1,0,63,22,68,1300.0000,1306.7233,41.705170,93,17,66,1300.0000,1293.2767,0,1,0.359935
2,2,1,0,1947,1,0,15,43,63,1300.0000,1309.6521,42.012257,63,22,47,1306.7233,1297.0712,1,1,0.631101
3,2,1,1,1947,2,0,63,22,47,1306.7233,1297.0712,40.692783,15,43,63,1300.0000,1309.6521,0,0,0.368899
4,3,1,0,1947,1,0,27,11,33,1300.0000,1279.6189,38.864048,103,4,50,1300.0000,1320.3811,1,0,0.640065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125779,63155,1,0,2015,100,1,18,5,82,1723.4149,1704.3949,60.309792,30,50,103,1790.9591,1809.9791,1,0,0.546572
125780,63156,1,0,2015,102,1,30,50,104,1809.9791,1813.6349,68.013329,18,5,91,1704.3949,1700.7391,1,1,0.765565
125781,63156,1,1,2015,101,1,18,5,91,1704.3949,1700.7391,60.010067,30,50,104,1809.9791,1813.6349,0,0,0.234435
125782,63157,1,0,2015,102,1,18,5,97,1700.7391,1692.0859,59.290245,30,50,105,1813.6349,1822.2881,1,0,0.481450


In [8]:
column_name = 'game_result'

unique_classes = data[column_name].unique()

num_classes = len(unique_classes)

print(f"The column '{column_name}' has {num_classes} classes.")

The column 'game_result' has 2 classes.


In [9]:
duplicate_count = data.duplicated().sum()

data = data.drop_duplicates()

print(f"Number of duplicate rows: {duplicate_count}")

print(f"Shape of DataFrame without duplicates: {data.shape}")

Number of duplicate rows: 0
Shape of DataFrame without duplicates: (125784, 20)


In [10]:

x = data.drop('game_result', axis=1)
y = data['game_result']


In [11]:

x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7, random_state=42)

my_model = LinearRegression()

my_model.fit(x_train,y_train)

LinearRegression()

In [12]:
y_predict=my_model.predict(x_train)

In [13]:

mse = mean_squared_error(y_train, y_predict)
mae = mean_absolute_error(y_train, y_predict)
r2 = r2_score(y_train, y_predict)

print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("R-squared:", r2)


Mean squared error: 0.026522140436408246
Mean absolute error: 0.11080994700567033
R-squared: 0.8939113322813378


In [14]:


from sklearn.preprocessing import PolynomialFeatures

degree = 3

poly_features = PolynomialFeatures(degree=degree, include_bias=False)

x_train_poly = poly_features.fit_transform(x_train)
x_test_poly = poly_features.transform(x_test)

model = LinearRegression()
model.fit(x_train_poly, y_train)

y_pred = model.predict(x_test_poly)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("R-squared:", r2)


Mean squared error: 0.0010928197762164527
Mean absolute error: 0.019877449446716107
R-squared: 0.9956286971232001


In [15]:
decision_tree = DecisionTreeClassifier()

decision_tree.fit(x_train, y_train)

y_pred_dt = decision_tree.predict(x_test)

accuracy = decision_tree.score(x_test, y_test)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix:\n", cm)


print("Classification Report:\n", classification_report(y_test, y_pred_dt))


Accuracy: 0.998622005511978
Confusion Matrix:
 [[18891    21]
 [   31 18793]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     18912
           1       1.00      1.00      1.00     18824

    accuracy                           1.00     37736
   macro avg       1.00      1.00      1.00     37736
weighted avg       1.00      1.00      1.00     37736



In [16]:

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(x_train, y_train)

y_pred_knn = knn.predict(x_test)

accuracy = knn.score(x_test, y_test)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix:\n", cm)

print("Classification Report:\n", classification_report(y_test, y_pred_knn))


Accuracy: 0.6398399406402374
Confusion Matrix:
 [[12096  6816]
 [ 6775 12049]]
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.64      0.64     18912
           1       0.64      0.64      0.64     18824

    accuracy                           0.64     37736
   macro avg       0.64      0.64      0.64     37736
weighted avg       0.64      0.64      0.64     37736



In [17]:

naive_bias = np.mean(y_train)

bias = y_pred - naive_bias

mean_bias = np.mean(bias)

print("Mean bias:", mean_bias)


Mean bias: -0.001415037863778946
